# This is a collection of BigQuery snippets

## Updating columns to modify text or PII

### normal column

In [ ]:
UPDATE `airnz-ga-bigquery.Misc_Exports.ga_sessions_20191005`
set geoNetwork.country = CASE
  WHEN regexp_contains(geoNetwork.country, 'Singapore') THEN 'SG'
else geoNetwork.country
 END
      WHERE
       geoNetwork.country is not null;  

### user/session scoped CD

In [ ]:
UPDATE `airnz-ga-bigquery.Misc_Exports.ga_sessions_20191005`
SET customDimensions = 
      ARRAY(
        SELECT AS STRUCT cd.index,
          CASE WHEN cd.index = 4 and regexp_contains(cd.value, 'americanexpress') THEN 'amex'
          ELSE cd.value
          END
        FROM UNNEST(customDimensions) AS cd
      ) 
WHERE TRUE

### hit-scoped column - pages, events, custom dimension

#### pagepath

In [ ]:
UPDATE `airnz-ga-bigquery.Misc_Exports.ga_sessions_20191005`
SET hits =
  ARRAY(
    SELECT AS STRUCT * REPLACE(
      (SELECT AS STRUCT page.* REPLACE(case when regexp_contains(page.pagepath, 'selectitinerary') then 'select_itinerary' else page.pagepath END AS pagepath)) AS page)
    FROM UNNEST(hits) hits
  )
WHERE TRUE;

#### events

In [ ]:
UPDATE `airnz-ga-bigquery.Misc_Exports.ga_sessions_20191005`
SET hits =
  ARRAY(
    SELECT AS STRUCT * REPLACE(
      (SELECT AS STRUCT eventInfo.* REPLACE(case when regexp_contains(eventInfo.eventCategory, 'fare finder') then 'fare_finder' else eventInfo.eventCategory END AS eventCategory)) AS eventInfo)
    FROM UNNEST(hits) hits
  )
WHERE TRUE;

#### custom dimension

able to include multiple custom dimensions using IN(). Otherwise define each rule for CD by adding more "WHEN" clauses

In [ ]:
UPDATE `airnz-ga-bigquery.Misc_Exports.ga_sessions_20191005`
SET hits = 
  ARRAY(
    SELECT AS STRUCT * REPLACE(
      ARRAY(
        SELECT AS STRUCT customdimensions.* REPLACE(
          CASE WHEN customdimensions.index IN (18,26) and regexp_contains(customdimensions.value, 'multistop|tas_man') THEN 'gotcha'  
     --   CASE WHEN customdimensions.index IN (18,26) then regexp_replace(customdimensions.value, '/(surname=)[^&\/\?]+/gi', 'redacted')
            ELSE value        
            END AS value)
        FROM UNNEST(customdimensions) customdimensions     
      ) AS customdimensions)
    FROM UNNEST(hits) hit
  ) 

WHERE TRUE 

### product scope column

In [ ]:
UPDATE `airnz-ga-bigquery.Misc_Exports.ga_sessions_20191006`

SET hits = 
  ARRAY(
    SELECT AS STRUCT * REPLACE(
      ARRAY(
        SELECT AS STRUCT product.* REPLACE(
          CASE WHEN regexp_contains(product.v2productCategory, 'ancillary') THEN 'ancillary_revenue'  
            ELSE v2productCategory END AS v2productCategory)
        FROM UNNEST(product) product
        
      ) AS product)
    FROM UNNEST(hits) hit
  ) 

WHERE TRUE  

### product scoped custom dimensions

In [ ]:
UPDATE
  `airnz-ga-bigquery.Misc_Exports.ga_sessions_20191005`
SET
  hits = ARRAY (
  SELECT
    AS STRUCT * REPLACE(
      ------------------------------
      ARRAY (
        ------------------------------
      SELECT
        AS STRUCT * REPLACE ( ARRAY(
          SELECT
            AS STRUCT customdimensions.* REPLACE(CASE
                WHEN customdimensions.index IN (27,29) AND REGEXP_CONTAINS(customdimensions.value, 'one way|fji') THEN 'gotcha' ELSE value
            END
              AS value)
          FROM
            UNNEST(customdimensions) customdimensions ) AS customdimensions)
      FROM
        UNNEST(product) product
        ------------------------------
        ) AS PRODUCT
      --------------------------------
      )
  FROM
    UNNEST(hits) hit )
WHERE
  TRUE

## Example of querying and joining different scoped dimensions and metrics into single source consolidated table

In [ ]:
CREATE TEMP FUNCTION
  customDimensionByIndex(indx INT64,
    arr ARRAY<STRUCT<index INT64,
    value STRING>>) AS ( (
    SELECT
      x.value
    FROM
      UNNEST(arr) x
    WHERE
      indx=x.index) );
WITH
  base_table AS (
  SELECT
    CONCAT(SUBSTR(date,0,4),'-',SUBSTR(date,5,2),'-',SUBSTR(date,7,2)) AS date,
    financialYear,
    financialQuarter,
    financialMonthNumber,
    financialWeekNumber,
    CASE
      WHEN financialWeekNumber IN ('0', '1', '2', '3', '4') THEN 'P01'
      WHEN financialWeekNumber IN ('5',
      '6',
      '7',
      '8') THEN 'P02'
      WHEN financialWeekNumber IN ('9', '10', '11', '12', '13') THEN 'P03'
      WHEN financialWeekNumber IN ('14',
      '15',
      '16',
      '17') THEN 'P04'
      WHEN financialWeekNumber IN ('18', '19', '20', '21') THEN 'P05'
      WHEN financialWeekNumber IN ('22',
      '23',
      '24',
      '25',
      '26') THEN 'P06'
      WHEN financialWeekNumber IN ('27', '28', '29', '30') THEN 'P07'
      WHEN financialWeekNumber IN ('31',
      '32',
      '33',
      '34') THEN 'P08'
      WHEN financialWeekNumber IN ('35', '36', '37', '38', '39') THEN 'P09'
      WHEN financialWeekNumber IN ('40',
      '41',
      '42',
      '43') THEN 'P10'
      WHEN financialWeekNumber IN ('44', '45', '46', '47') THEN 'P11'
      WHEN financialWeekNumber IN ('48',
      '49',
      '50',
      '51',
      '52') THEN 'P12'
    ELSE
    NULL
  END
    AS PurchasePeriod,
    channelGrouping2,
    deviceCategory,
    storefront,
    COUNT(DISTINCT sessions) AS sessions,
    COUNT(DISTINCT mobileSessions) AS mobileSessions,
    COUNT(DISTINCT fullVisitorId) AS dailyUsers,
    SUM(bounces) AS bounces,
    COUNT(DISTINCT newSessions) AS newSessions,
    COUNT(DISTINCT uniquePageViews) AS uniquePageViews,
    SUM(sessionDuration) AS sessionDuration,
    COUNT(DISTINCT signedInSessions) AS signedInSessions,
    COUNT(DISTINCT vBookTotalSessions ) AS vBookTotalSessions,
    COUNT(DISTINCT vbookSearchPageSessions ) AS vbookSearchPageSessions,
    COUNT(DISTINCT vBookFlightSearchSession ) AS vBookFlightSearchSession,
    COUNT(DISTINCT vBookTravellerDetailsSession ) AS vBookTravellerDetailsSession,
    COUNT(DISTINCT vBookExtrasSession ) AS vBookExtrasSession,
    COUNT(DISTINCT vBookSeatSelectSession) AS vbookSeatSelectSession,
    COUNT(DISTINCT vbookPurchaseTicketsSession ) AS vbookPurchaseTicketsSession,
    COUNT(DISTINCT vbookBookingConfirmationSession ) AS vbookBookingConfirmationSession,
    SUM(totalRevenueNzd) AS totalRevenueNzd,
    SUM(totalPaidSeatsQty) AS totalPaidSeatsQty,
    SUM(vBookFLightsQty) AS vBookFLightsQty,
    SUM(vbookFlightsRevenueNZD) AS vbookFlightsRevenueNZD,
    SUM(vbookFlightsRevenueLocal) AS vbookFlightsRevenueLocal,
    SUM(totalPaidSeatsRevenueNZD) AS totalPaidSeatsRevenueNZD,
    SUM(totalPaidSeatsRevenueLocal) AS totalPaidSeatsRevenueLocal,
    SUM(totalExtraBagsQty) AS totalExtraBagsQty,
    SUM(totalExtraBagsRevenue) AS totalExtraBagsRevenue,
    SUM(totalSkycouchQty) AS totalSkycouchQty,
    SUM(totalSkycouchRevenueNZD) AS totalSkycouchRevenueNZD,
    SUM(totalSkycouchRevenueLocal) AS totalSkycouchRevenueLocal,
    COUNT(DISTINCT promoCodeBookingSessions ) AS promoCodeBookingSessions,
    SUM(vbookRevenueNzd) AS vbookRevenueNzd,
    SUM(vbookRevenueLocal) AS vbookRevenueLocal,
    SUM(vbookPaidSeatsQty) AS vbookPaidSeatsQty,
    SUM(fareholdQty) AS fareholdQty,
    SUM(vbookExtraBagQty) AS vbookExtraBagQty,
    SUM(vbookSkycouchQty) AS vbookSkycouchQty,
    SUM(vbookInsuranceQty) AS vbookInsuranceQty,
    SUM(vbookCarRentalQty) AS vbookCarRentalQty,
    SUM(vbookCarbonOffsetQty) AS vbookCarbonOffsetQty,
    SUM(vbookFlexipayFeeQty) AS vbookFlexipayFeeQty,
    SUM(vbookDirectAncillaryRevenueNzd) AS vbookDirectAncillaryRevenueNzd,
    SUM(vbookIndirectAncillaryRevenueNzd) AS vbookIndirectAncillaryRevenueNzd,
    SUM(vbookDirectAncillaryRevenueLocal) AS vbookDirectAncillaryRevenueLocal,
    SUM(vbookIndirectAncillaryRevenueLocal) AS vbookIndirectAncillaryRevenueLocal
  FROM (
    SELECT
      date,
      CASE
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (7,  8,  9) THEN CONCAT(CAST( EXTRACT(YEAR  FROM (DATE_ADD(PARSE_DATE('%Y%m%d',  date), INTERVAL 1 YEAR) )) AS string ))
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (10,
        11,
        12) THEN CONCAT(CAST( EXTRACT(YEAR
          FROM (DATE_ADD(PARSE_DATE('%Y%m%d',
                  date), INTERVAL 1 YEAR) )) AS string ))
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (1,  2,  3) THEN CONCAT(CAST( EXTRACT(YEAR  FROM (DATE_ADD(PARSE_DATE('%Y%m%d',  date), INTERVAL 0 YEAR) )) AS string ))
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (4,
        5,
        6) THEN CONCAT(CAST( EXTRACT(YEAR
          FROM (DATE_ADD(PARSE_DATE('%Y%m%d',
                  date), INTERVAL 0 YEAR) )) AS string ))
      ELSE
      'NA'
    END
      AS financialYear,
      CASE
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (7,  8,  9) THEN 'Q1'
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (10,
        11,
        12) THEN 'Q2'
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (1,  2,  3) THEN 'Q3'
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (4,
        5,
        6) THEN 'Q4'
    END
      AS financialQuarter,
      CASE
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (7, 8, 9, 10, 11, 12) THEN CAST (EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date))AS INT64) - 6
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (1,
        2,
        3,
        4,
        5,
        6) THEN CAST (EXTRACT(MONTH
        FROM
          PARSE_DATE('%Y%m%d',
            date)) AS INT64) + 6
    END
      AS financialMonthNumber,
      CASE
        WHEN date IN ('20160701',  '20160702',  '20160703',  '20170701',  '20170702',  '20180701',  '20200701',  '20200702',  '20200703',  '20200704',  '20200705',  '20210701',  '20210702',  '20210703',  '20210704') THEN '0'
        WHEN CAST(EXTRACT(ISOWEEK
        FROM
          PARSE_DATE('%Y%m%d',
            date)) AS INT64) <=26 THEN CAST(EXTRACT(ISOWEEK
        FROM
          PARSE_DATE('%Y%m%d',
            date)) +26 AS STRING)
        WHEN CAST(EXTRACT(ISOWEEK  FROM  PARSE_DATE('%Y%m%d',  date)) AS INT64) >26 THEN CAST(EXTRACT(ISOWEEK  FROM  PARSE_DATE('%Y%m%d',  date)) -26 AS STRING)
      ELSE
      '0'
    END
      AS financialWeekNumber,
      channelGrouping,
      CASE
        WHEN REGEXP_CONTAINS(source,r'airnz-mobile-app') THEN 'app referral'
      ELSE
      channelGrouping
    END
      AS channelGrouping2,
      deviceCategory,
      CASE
        WHEN REGEXP_CONTAINS(source,r'airnz-mobile-app') THEN 'app referral'
        WHEN REGEXP_CONTAINS(deviceCategory,r'mobile') THEN 'mobile web'
      ELSE
      deviceCategory
    END
      AS deviceCategory2,
      CASE
        WHEN REGEXP_CONTAINS(hostname,r'flybuysflights.airnewzealand.co.nz') THEN 'FLYBUYS-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.nz') THEN 'AGENT-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.com.au') THEN 'AGENT-AU'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.jp') THEN 'AGENT-JP'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.com.sg') THEN 'AGENT-SG'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent-ar.com') THEN 'AGENT-AR'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.com') THEN 'AGENT-US'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.de') THEN 'AGENT-DE'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.uk') THEN 'AGENT-UK'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.in') THEN 'AGENT-IN'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.ca|www.airnzagent.ca') THEN 'AGENT-CA'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.fr|www.airnzagent.fr') THEN 'AGENT-FR'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.hk|www.airnzagent.hk') THEN 'AGENT-HK'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.tw|www.airnzagent.tw') THEN 'AGENT-TW'
        WHEN REGEXP_CONTAINS(hostname,r'www.tandemtravel.co.nz') THEN 'TANDEM'
        WHEN REGEXP_CONTAINS(hostname,r'govtbookings.airnewzealand.co.nz') THEN 'GOVT-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'.*grabaseat.co.nz$') THEN 'GAS-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'(airn(z|ewzealand)|taxi|airpoints).co.nz') THEN 'NZ'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.com.au$') THEN 'AU'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.ca$') THEN 'CA'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.co.uk$') THEN 'UK'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.eu$') THEN 'EU'
        WHEN REGEXP_CONTAINS(hostname,r'(.jp|jp.airnewzealand.com)$') THEN 'JP'
        WHEN REGEXP_CONTAINS(hostname,r'.jp.airnewzealand.com$') THEN 'JP'
        WHEN REGEXP_CONTAINS(hostname,r'(.cn|cn.airnewzealand.com)$') THEN 'CN'
        WHEN REGEXP_CONTAINS(hostname,r'.*.hk$') THEN 'HK'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.com.sg$') THEN 'SG'
        WHEN REGEXP_CONTAINS(hostname, r'.my') THEN 'MY'
        WHEN REGEXP_CONTAINS(hostname, r'.th') THEN 'TH'
        WHEN REGEXP_CONTAINS(hostname,r'.vn$') THEN 'VN'
        WHEN REGEXP_CONTAINS(hostname,r'.in$') THEN 'IN'
        WHEN REGEXP_CONTAINS(hostname,r'.id$') THEN 'ID'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand-ar.com$') THEN 'AR'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand-br.com$') THEN 'BR'
        WHEN REGEXP_CONTAINS(hostname,r'.kr$') THEN 'KR'
        WHEN REGEXP_CONTAINS(hostname,r'(.tw|tw.airnewzealand.com)$') THEN 'TW'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.fr$') THEN 'FR'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.de$') THEN 'DE'
        WHEN REGEXP_CONTAINS(hostname,r'.es$') THEN 'ES'
        WHEN REGEXP_CONTAINS(hostname,r'.it$') THEN 'IT'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.pf$') THEN 'PF'
        WHEN REGEXP_CONTAINS(hostname,r'.pacificislands') THEN 'PI'
        WHEN REGEXP_CONTAINS(hostname,r'.mx$') THEN 'MX'
        WHEN REGEXP_CONTAINS(hostname,r'.com.co$') THEN 'CO'
        WHEN REGEXP_CONTAINS(hostname,r'^airnz.custhelp.com$') THEN 'FAQ-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'^airnzau.custhelp.com$') THEN 'FAQ-AU'
        WHEN REGEXP_CONTAINS(hostname,r'^airnzcaus.custhelp.com$') THEN 'FAQ-US'
        WHEN REGEXP_CONTAINS(hostname,r'^airnzukeu.custhelp.com$') THEN 'FAQ-EU'
        WHEN REGEXP_CONTAINS(hostname,r'^wifi') THEN 'WIFI'
        WHEN REGEXP_CONTAINS(hostname,r'.airnewzealand.com$')
      AND NOT REGEXP_CONTAINS(hostname,r'au|sg|hk|jp|pacificislands|cn|eu|vn|.com.co|carrental') THEN 'US'
        WHEN REGEXP_CONTAINS(hostname,r'carrental') THEN 'CAR RENTAL'
      ELSE
      'OTHER'
    END
      AS storefront,
      visitid AS sessions,
      CASE
        WHEN deviceCategory IN ('mobile') THEN visitid
      ELSE
      NULL
    END
      AS mobileSessions,
      fullVisitorId,
      CASE
        WHEN hitNumber = first_interaction THEN bounces
      ELSE
      0
    END
      AS bounces,
      newVisits AS newSessions,
      session_id AS uniquePageViews,
      CASE
        WHEN hitnumber = first_hit THEN sessionDuration
      ELSE
      NULL
    END
      AS sessionDuration,
      signedInSessions,
      CASE
        WHEN REGEXP_CONTAINS(pagePath, r'^/vbook/') THEN visitid
      ELSE
      NULL
    END
      AS vBookTotalSessions,
      CASE
        WHEN REGEXP_CONTAINS(pagePath,r'^/vbook/actions(|/mobi)(/search|/ext-search)') THEN visitid
      ELSE
      NULL
    END
      AS vbookSearchPageSessions,
      CASE
        WHEN REGEXP_CONTAINS(pagePath, r'^/vbook/actions/(selectflights|selectitinerary|(mobi/|)createitinerary)') THEN visitid
      ELSE
      NULL
    END
      AS vBookFlightSearchSession,
      CASE
        WHEN REGEXP_CONTAINS(pagePath, r'^/vbook/actions/(mobi/|)travellerdetails') THEN visitid
      ELSE
      NULL
    END
      AS vBookTravellerDetailsSession,
      CASE
        WHEN REGEXP_CONTAINS(pagePath, r'^/vbook/actions/(mobi/|)extras') THEN visitid
      ELSE
      NULL
    END
      AS vBookExtrasSession,
      CASE
        WHEN REGEXP_CONTAINS(pagePath, r'^/vbook/actions(/|/mobi/)(seatselect|select-your-seats)') THEN visitid
      ELSE
      NULL
    END
      AS vBookSeatSelectSession,
      CASE
        WHEN REGEXP_CONTAINS(pagePath, r'^/vbook/actions/(mobi/|)purchasetickets') THEN visitid
      ELSE
      NULL
    END
      AS vBookPurchaseTicketsSession,
      CASE
        WHEN REGEXP_CONTAINS(pagePath, r'^/vbook/actions/(mobi/|)bookingconfirmation') THEN visitid
      ELSE
      NULL
    END
      AS vbookBookingConfirmationSession,
      transactionId,
      SUM(totalRevenueNZD) AS totalRevenueNZD,
      SUM (vBookFLightsQty) AS vBookFLightsQty,
      SUM(vbookFlightsRevenueNZD) AS vbookFlightsRevenueNZD,
      SUM(vbookFlightsRevenueLocal) AS vbookFlightsRevenueLocal,
      SUM (totalPaidSeatsQty) AS totalPaidSeatsQty,
      SUM(totalPaidSeatsRevenueNZD) AS totalPaidSeatsRevenueNZD,
      SUM(totalPaidSeatsRevenueLocal) AS totalPaidSeatsRevenueLocal,
      SUM(totalExtraBagsQty) AS totalExtraBagsQty,
      SUM(totalExtraBagsRevenue) AS totalExtraBagsRevenue,
      SUM (totalSkycouchQty) AS totalSkycouchQty,
      SUM(totalSkycouchRevenueNZD) AS totalSkycouchRevenueNZD,
      SUM(totalSkycouchRevenueLocal) AS totalSkycouchRevenueLocal,
      promoCodeBookingSessions,
      SUM(vbookRevenueNzd) AS vbookRevenueNzd,
      SUM(vbookRevenueLocal) AS vbookRevenueLocal,
      SUM(vbookPaidSeatsQty) AS vbookPaidSeatsQty,
      SUM (fareholdQty) AS fareholdQty,
      SUM (vbookExtraBagQty) AS vbookExtraBagQty,
      SUM (vbookSkycouchQty) AS vbookSkycouchQty,
      SUM (vbookInsuranceQty) AS vbookInsuranceQty,
      SUM (vbookCarRentalQty) AS vbookCarRentalQty,
      SUM (vbookCarbonOffsetQty) AS vbookCarbonOffsetQty,
      SUM (vbookFlexipayFeeQty) AS vbookFlexipayFeeQty,
      SUM(vbookDirectAncillaryRevenueNzd) AS vbookDirectAncillaryRevenueNzd,
      SUM(vbookIndirectAncillaryRevenueNzd) AS vbookIndirectAncillaryRevenueNzd,
      SUM(
      IF
        (REGEXP_CONTAINS(pagePath,r'^/vbook/')
          AND REGEXP_CONTAINS(eventAction,r'purchase'),
          transactionTax,
          NULL))/1000000 AS vbookTaxesFeesRevenueNzd,
      SUM(vbookDirectAncillaryRevenueLocal) AS vbookDirectAncillaryRevenueLocal,
      SUM(vbookIndirectAncillaryRevenueLocal) AS vbookIndirectAncillaryRevenueLocal,
      SUM(
      IF
        (REGEXP_CONTAINS(pagePath,r'^/vbook/')
          AND REGEXP_CONTAINS(eventAction,r'purchase'),
          localTransactionTax,
          NULL))/1000000 AS vbookTaxesFeesRevenueLocal
    FROM (
      SELECT
        date,
        fullVisitorId,
        CONCAT(fullVisitorId,'-',CAST(visitId AS STRING)) AS visitID,
        h.page.pagePath,
        channelGrouping,
        trafficSource.source,
        device.deviceCategory,
        device.LANGUAGE AS LANGUAGE,
        h.isInteraction,
        h.hitnumber,
        h.page.hostname,
        totals.visits AS visits,
        totals.bounces,
        CASE
          WHEN totals.newVisits IS NOT NULL THEN fullvisitorid
        ELSE
        NULL
      END
        AS newVisits,
        MIN(h.hitNumber) OVER (PARTITION BY fullVisitorId, visitStartTime) AS first_hit,
        MIN(
        IF
          (h.isInteraction IS NOT NULL,
            h.hitNumber,
            0)) OVER (PARTITION BY fullVisitorId, visitStartTime) AS first_interaction,
        CASE
          WHEN h.type = 'PAGE' THEN CONCAT(fullVisitorId, CAST(visitid AS STRING))
        ELSE
        ''
      END
        AS session_id,
        totals.timeonsite AS sessionDuration,
        CASE
          WHEN REGEXP_CONTAINS(customDimensionByIndex(72, h.customDimensions),r'.*') AND LENGTH(customDimensionByIndex(72, h.customDimensions)) > 4 THEN CONCAT(fullVisitorId, CAST(visitid AS STRING))
        ELSE
        NULL
      END
        AS signedInSessions,
        h.transaction.transactionId,
        SUM(p.productRevenue)/POW(10,6) AS totalRevenueNZD,
        p.v2ProductCategory,
        h.eventInfo.eventAction,
        p.productQuantity,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(v2ProductCategory,r'^flight$') AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS vBookFLightsQty,
        p.productRevenue,
        SUM (CASE
            WHEN REGEXP_CONTAINS(v2ProductCategory,r'^flight$') AND REGEXP_CONTAINS(h.page.pagePath, r'^/vbook/') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN p.productRevenue
          ELSE
          NULL
        END
          )/1000000 AS vbookFlightsRevenueNZD,
        SUM (CASE
            WHEN REGEXP_CONTAINS(v2ProductCategory,r'^flight$') AND REGEXP_CONTAINS(h.page.pagePath, r'^/vbook/') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN p.localproductRevenue
          ELSE
          NULL
        END
          )/1000000 AS vbookFlightsRevenueLocal,
        SUM (CASE
            WHEN REGEXP_CONTAINS(h.page.pagePath, r'^/vbook/') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN p.productRevenue
          ELSE
          NULL
        END
          )/1000000 AS vbookRevenueNzd,
        SUM (CASE
            WHEN REGEXP_CONTAINS(h.page.pagePath, r'^/vbook/') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN p.localProductRevenue
          ELSE
          NULL
        END
          )/1000000 AS vbookRevenueLocal,
        p.localProductRevenue,
        p.v2ProductName,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(p.v2ProductName,r'seat select') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') AND REGEXP_CONTAINS(customDimensionByIndex(55, p.customDimensions),r'paid') THEN productQuantity
          ELSE
          NULL
        END
          ) AS totalPaidSeatsQty,
        SUM(
          CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(p.v2ProductName, 'extra bags') AND REGEXP_CONTAINS(h.eventinfo.eventAction,r'purchase') THEN p.productQuantity
          ELSE
          NULL
        END
          ) AS totalExtraBagsQty,
        SUM (CASE
            WHEN REGEXP_CONTAINS(v2ProductName,r'extra bags') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN p.productRevenue
          ELSE
          NULL
        END
          )/1000000 AS totalExtraBagsRevenue,
        SUM (
        IF
          (REGEXP_CONTAINS(p.v2ProductName,r'seat select')
            AND REGEXP_CONTAINS(customDimensionByIndex(55,
                p.customDimensions),r'paid')
            AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase'),
            productRevenue,
            NULL))/1000000 AS totalPaidSeatsRevenueNZD,
        SUM (
        IF
          ( REGEXP_CONTAINS(p.v2ProductName,r'seat select')
            AND REGEXP_CONTAINS(customDimensionByIndex(55,
                p.customDimensions),r'paid')
            AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase'),
            localProductRevenue,
            NULL))/1000000 AS totalPaidSeatsRevenueLocal,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(v2ProductName,r'skycouch') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS totalSkycouchQty,
        SUM (CASE
            WHEN REGEXP_CONTAINS(v2ProductName,r'skycouch') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN p.productRevenue
          ELSE
          NULL
        END
          )/1000000 AS totalSkycouchRevenueNZD,
        SUM (CASE
            WHEN REGEXP_CONTAINS(v2ProductName,r'skycouch') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN p.localproductRevenue
          ELSE
          NULL
        END
          )/1000000 AS totalSkycouchRevenueLocal,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(v2ProductName,r'fare hold') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS fareholdQty,
        CASE
          WHEN REGEXP_CONTAINS(h.page.pagePath, r'/vbook/actions/bookingconfirmation') AND REGEXP_CONTAINS(( SELECT  value  FROM  h.customDimensions  WHERE  index = 25),r'.*') THEN CONCAT(fullVisitorId,'-',CAST(visitid AS STRING))
        ELSE
        NULL
      END
        AS promoCodeBookingSessions,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(p.v2ProductName,r'seat select') AND REGEXP_CONTAINS(customDimensionByIndex(55, p.customDimensions),r'paid') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS vbookPaidSeatsQty,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(v2ProductName,r'extra bags') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS vbookExtraBagQty,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(v2ProductName,r'skycouch') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS vbookSkycouchQty,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(v2ProductName,r'insurance') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS vbookInsuranceQty,
        SUM(
          CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(v2ProductName,r'^car$') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS vbookCarRentalQty,
        SUM(
          CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(v2ProductName,r'carbon offset') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS vbookCarbonOffsetQty,
        SUM(CASE
            WHEN h.eCommerceAction.action_type = '6' AND REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/') AND REGEXP_CONTAINS(v2ProductName,r'flexipay fee') AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase') THEN productQuantity
          ELSE
          NULL
        END
          ) AS vbookFlexipayFeeQty,
        SUM (
        IF
          (REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/')
            AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase')
            AND REGEXP_CONTAINS(p.v2ProductCategory,r'^ancillary$')
            AND REGEXP_CONTAINS(customDimensionByIndex(57,
                p.customDimensions),r'^direct$'),
            p.productRevenue,
            NULL))/1000000 AS vbookDirectAncillaryRevenueNzd,
        SUM (
        IF
          (REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/')
            AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase')
            AND REGEXP_CONTAINS(p.v2ProductCategory,r'^ancillary$')
            AND REGEXP_CONTAINS(customDimensionByIndex(57,
                p.customDimensions),r'^indirect$'),
            p.productRevenue,
            NULL))/1000000 AS vbookIndirectAncillaryRevenueNzd,
        h.transaction.transactionTax,
        SUM (
        IF
          (REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/')
            AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase')
            AND REGEXP_CONTAINS(p.v2ProductCategory,r'^ancillary$')
            AND REGEXP_CONTAINS(customDimensionByIndex(57,
                p.customDimensions),r'^direct$'),
            p.localProductRevenue,
            NULL))/1000000 AS vbookDirectAncillaryRevenueLocal,
        SUM (
        IF
          (REGEXP_CONTAINS(h.page.pagePath,r'^/vbook/')
            AND REGEXP_CONTAINS(h.eventInfo.eventAction,r'purchase')
            AND REGEXP_CONTAINS(p.v2ProductCategory,r'^ancillary$')
            AND REGEXP_CONTAINS(customDimensionByIndex(57,
                p.customDimensions),r'^indirect$'),
            p.localProductRevenue,
            NULL))/1000000 AS vbookIndirectAncillaryRevenueLocal,
        h.transaction.localTransactionTax
      FROM
        `airnz-ga-bigquery.125557395.ga_sessions_*`,
        UNNEST(hits) AS h
      LEFT JOIN
        UNNEST (product) AS p
      WHERE
        _TABLE_SUFFIX BETWEEN '20190804'
        AND '20190804'
      GROUP BY
        date,
        fullVisitorId,
        visitId,
        isInteraction,
        visitStartTime,
        hitnumber,
        hostname,
        pagePath,
        channelGrouping,
        source,
        deviceCategory,
        LANGUAGE,
        hostname,
        totals.visits,
        totals.timeonsite,
        totals.bounces,
        totals.newVisits,
        session_id,
        h.transaction.transactionId,
        h.transaction.localTransactionRevenue,
        h.transaction.transactionRevenue,
        p.v2ProductCategory,
        h.eventInfo.eventAction,
        p.productQuantity,
        p.productRevenue,
        p.localProductRevenue,
        p.v2ProductName,
        promoCodeBookingSessions,
        h.transaction.transactionTax,
        h.transaction.localTransactionTax,
        signedInSessions )
    GROUP BY
      date,
      hostname,
      first_interaction,
      bounces,
      uniquePageViews,
      financialYear,
      financialQuarter,
      financialMonthNumber,
      financialWeekNumber,
      vbookSearchPageSessions,
      channelGrouping,
      channelGrouping2,
      fullVisitorId,
      hitNumber,
      first_hit,
      visits,
      transactionId,
      deviceCategory,
      deviceCategory2,
      storefront,
      visitId,
      sessionDuration,
      vBookTotalSessions,
      vBookFlightSearchSession,
      vBookTravellerDetailsSession,
      vBookExtrasSession,
      vBookSeatSelectSession,
      vBookPurchaseTicketsSession,
      vbookBookingConfirmationSession,
      promoCodeBookingSessions,
      signedInSessions,
      newSessions )
  GROUP BY
    date,
    financialYear,
    financialQuarter,
    financialMonthNumber,
    financialWeekNumber,
    channelGrouping2,
    deviceCategory,
    PurchasePeriod,
    storefront),
    
    
  hitscope_table AS (
  SELECT
    CONCAT(SUBSTR(date,0,4),'-',SUBSTR(date,5,2),'-',SUBSTR(date,7,2)) AS date_hitscope,
    financialYear financialYear_hitscope,
    financialQuarter financialQuarter_hitscope,
    financialMonthNumber financialMonthNumber_hitscope,
    financialWeekNumber financialWeekNumber_hitscope,
    CASE
      WHEN financialWeekNumber IN ('0', '1', '2', '3', '4') THEN 'P01'
      WHEN financialWeekNumber IN ('5',
      '6',
      '7',
      '8') THEN 'P02'
      WHEN financialWeekNumber IN ('9', '10', '11', '12', '13') THEN 'P03'
      WHEN financialWeekNumber IN ('14',
      '15',
      '16',
      '17') THEN 'P04'
      WHEN financialWeekNumber IN ('18', '19', '20', '21') THEN 'P05'
      WHEN financialWeekNumber IN ('22',
      '23',
      '24',
      '25',
      '26') THEN 'P06'
      WHEN financialWeekNumber IN ('27', '28', '29', '30') THEN 'P07'
      WHEN financialWeekNumber IN ('31',
      '32',
      '33',
      '34') THEN 'P08'
      WHEN financialWeekNumber IN ('35', '36', '37', '38', '39') THEN 'P09'
      WHEN financialWeekNumber IN ('40',
      '41',
      '42',
      '43') THEN 'P10'
      WHEN financialWeekNumber IN ('44', '45', '46', '47') THEN 'P11'
      WHEN financialWeekNumber IN ('48',
      '49',
      '50',
      '51',
      '52') THEN 'P12'
    ELSE
    NULL
  END
    AS PurchasePeriod_hitscope,
    channelGrouping2 channelGrouping2_hitscope,
    deviceCategory deviceCategory_hitscope,
    storefront storefront_hitscope,
    SUM(users) users,
    SUM(unique_pageview_id) unique_pageview_id,
    SUM(transactiontax) vbookTaxesFeesRevenueNzd,
    SUM(localTransactionTax) vbookTaxesFeesRevenueLocal,
    SUM(localTransactionRevenue) localTransactionRevenue,
    SUM(transactions) transactions
  FROM (
    SELECT
      date,
      hostname,
      CASE
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (7,  8,  9) THEN CONCAT(CAST( EXTRACT(YEAR  FROM (DATE_ADD(PARSE_DATE('%Y%m%d',  date), INTERVAL 1 YEAR) )) AS string ))
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (10,
        11,
        12) THEN CONCAT(CAST( EXTRACT(YEAR
          FROM (DATE_ADD(PARSE_DATE('%Y%m%d',
                  date), INTERVAL 1 YEAR) )) AS string ))
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (1,  2,  3) THEN CONCAT(CAST( EXTRACT(YEAR  FROM (DATE_ADD(PARSE_DATE('%Y%m%d',  date), INTERVAL 0 YEAR) )) AS string ))
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (4,
        5,
        6) THEN CONCAT(CAST( EXTRACT(YEAR
          FROM (DATE_ADD(PARSE_DATE('%Y%m%d',
                  date), INTERVAL 0 YEAR) )) AS string ))
      ELSE
      'NA'
    END
      AS financialYear,
      CASE
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (7,  8,  9) THEN 'Q1'
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (10,
        11,
        12) THEN 'Q2'
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (1,  2,  3) THEN 'Q3'
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (4,
        5,
        6) THEN 'Q4'
    END
      AS financialQuarter,
      CASE
        WHEN EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date)) IN (7, 8, 9, 10, 11, 12) THEN CAST (EXTRACT(MONTH  FROM  PARSE_DATE('%Y%m%d',  date))AS INT64) - 6
        WHEN EXTRACT(MONTH
      FROM
        PARSE_DATE('%Y%m%d',
          date)) IN (1,
        2,
        3,
        4,
        5,
        6) THEN CAST (EXTRACT(MONTH
        FROM
          PARSE_DATE('%Y%m%d',
            date)) AS INT64) + 6
    END
      AS financialMonthNumber,
      CASE
        WHEN date IN ('20160701',  '20160702',  '20160703',  '20170701',  '20170702',  '20180701',  '20200701',  '20200702',  '20200703',  '20200704',  '20200705',  '20210701',  '20210702',  '20210703',  '20210704') THEN '0'
        WHEN CAST(EXTRACT(ISOWEEK
        FROM
          PARSE_DATE('%Y%m%d',
            date)) AS INT64) <=26 THEN CAST(EXTRACT(ISOWEEK
        FROM
          PARSE_DATE('%Y%m%d',
            date)) +26 AS STRING)
        WHEN CAST(EXTRACT(ISOWEEK  FROM  PARSE_DATE('%Y%m%d',  date)) AS INT64) >26 THEN CAST(EXTRACT(ISOWEEK  FROM  PARSE_DATE('%Y%m%d',  date)) -26 AS STRING)
      ELSE
      '0'
    END
      AS financialWeekNumber,
      channelGrouping,
      CASE
        WHEN REGEXP_CONTAINS(source,r'airnz-mobile-app') THEN 'app referral'
      ELSE
      channelGrouping
    END
      AS channelGrouping2,
      deviceCategory,
      CASE
        WHEN REGEXP_CONTAINS(source,r'airnz-mobile-app') THEN 'app referral'
        WHEN REGEXP_CONTAINS(deviceCategory,r'mobile') THEN 'mobile web'
      ELSE
      deviceCategory
    END
      AS deviceCategory2,
      CASE
        WHEN REGEXP_CONTAINS(hostname,r'flybuysflights.airnewzealand.co.nz') THEN 'FLYBUYS-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.nz') THEN 'AGENT-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.com.au') THEN 'AGENT-AU'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.jp') THEN 'AGENT-JP'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.com.sg') THEN 'AGENT-SG'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent-ar.com') THEN 'AGENT-AR'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.com') THEN 'AGENT-US'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.de') THEN 'AGENT-DE'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.uk') THEN 'AGENT-UK'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.in') THEN 'AGENT-IN'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.ca|www.airnzagent.ca') THEN 'AGENT-CA'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.fr|www.airnzagent.fr') THEN 'AGENT-FR'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.hk|www.airnzagent.hk') THEN 'AGENT-HK'
        WHEN REGEXP_CONTAINS(hostname,r'www.airnzagent.co.tw|www.airnzagent.tw') THEN 'AGENT-TW'
        WHEN REGEXP_CONTAINS(hostname,r'www.tandemtravel.co.nz') THEN 'TANDEM'
        WHEN REGEXP_CONTAINS(hostname,r'govtbookings.airnewzealand.co.nz') THEN 'GOVT-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'.*grabaseat.co.nz$') THEN 'GAS-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'(airn(z|ewzealand)|taxi|airpoints).co.nz') THEN 'NZ'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.com.au$') THEN 'AU'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.ca$') THEN 'CA'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.co.uk$') THEN 'UK'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.eu$') THEN 'EU'
        WHEN REGEXP_CONTAINS(hostname,r'(.jp|jp.airnewzealand.com)$') THEN 'JP'
        WHEN REGEXP_CONTAINS(hostname,r'.jp.airnewzealand.com$') THEN 'JP'
        WHEN REGEXP_CONTAINS(hostname,r'(.cn|cn.airnewzealand.com)$') THEN 'CN'
        WHEN REGEXP_CONTAINS(hostname,r'.*.hk$') THEN 'HK'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.com.sg$') THEN 'SG'
        WHEN REGEXP_CONTAINS(hostname, r'.my') THEN 'MY'
        WHEN REGEXP_CONTAINS(hostname, r'.th') THEN 'TH'
        WHEN REGEXP_CONTAINS(hostname,r'.vn$') THEN 'VN'
        WHEN REGEXP_CONTAINS(hostname,r'.in$') THEN 'IN'
        WHEN REGEXP_CONTAINS(hostname,r'.id$') THEN 'ID'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand-ar.com$') THEN 'AR'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand-br.com$') THEN 'BR'
        WHEN REGEXP_CONTAINS(hostname,r'.kr$') THEN 'KR'
        WHEN REGEXP_CONTAINS(hostname,r'(.tw|tw.airnewzealand.com)$') THEN 'TW'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.fr$') THEN 'FR'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.de$') THEN 'DE'
        WHEN REGEXP_CONTAINS(hostname,r'.es$') THEN 'ES'
        WHEN REGEXP_CONTAINS(hostname,r'.it$') THEN 'IT'
        WHEN REGEXP_CONTAINS(hostname,r'.*airnewzealand.pf$') THEN 'PF'
        WHEN REGEXP_CONTAINS(hostname,r'.pacificislands') THEN 'PI'
        WHEN REGEXP_CONTAINS(hostname,r'.mx$') THEN 'MX'
        WHEN REGEXP_CONTAINS(hostname,r'.com.co$') THEN 'CO'
        WHEN REGEXP_CONTAINS(hostname,r'^airnz.custhelp.com$') THEN 'FAQ-NZ'
        WHEN REGEXP_CONTAINS(hostname,r'^airnzau.custhelp.com$') THEN 'FAQ-AU'
        WHEN REGEXP_CONTAINS(hostname,r'^airnzcaus.custhelp.com$') THEN 'FAQ-US'
        WHEN REGEXP_CONTAINS(hostname,r'^airnzukeu.custhelp.com$') THEN 'FAQ-EU'
        WHEN REGEXP_CONTAINS(hostname,r'^wifi') THEN 'WIFI'
        WHEN REGEXP_CONTAINS(hostname,r'.airnewzealand.com$')
      AND NOT REGEXP_CONTAINS(hostname,r'au|sg|hk|jp|pacificislands|cn|eu|vn|.com.co|carrental') THEN 'US'
        WHEN REGEXP_CONTAINS(hostname,r'carrental') THEN 'CAR RENTAL'
      ELSE
      'OTHER'
    END
      AS storefront,
      COUNT (DISTINCT fullvisitorID) AS users,
      COUNT(DISTINCT session_id) AS unique_pageview_id,
      SUM(transactionTax)/POW(10,6) transactiontax,
      SUM(localTransactionTax)/POW(10,6) localTransactionTax,
      SUM(localTransactionRevenue) localTransactionRevenue,
      COUNT(transactions) transactions
    FROM (
      SELECT
        date,
        fullVisitorId,
        visitStartTime,
        h.page.hostname,
        h.hitnumber,
        channelGrouping,
        trafficSource.source,
        device.deviceCategory,
        totals.visits AS visits,
        CASE
          WHEN h.eCommerceAction.action_type = '6' THEN h.transaction.transactionId
        ELSE
        NULL
      END
        AS transactions,
        SUM(h.transaction.localTransactionRevenue)/POW(10,6) AS localTransactionRevenue,
        MIN(h.hitNumber) OVER (PARTITION BY fullVisitorId, visitStartTime) AS first_hit,
        CASE
          WHEN h.type = 'PAGE' THEN CONCAT(fullVisitorId, CAST(visitStartTime AS STRING))
        ELSE
        ''
      END
        AS session_id,
        SUM(CASE
            WHEN REGEXP_CONTAINS(h.page.pagePath, '^/vbook/') AND REGEXP_CONTAINS(h.eventInfo.eventAction, 'purchase') THEN h.transaction.transactiontax
          ELSE
          NULL
        END
          ) AS transactionTax,
        SUM(
          CASE
            WHEN REGEXP_CONTAINS(h.page.pagePath, '^/vbook/') AND REGEXP_CONTAINS(h.eventInfo.eventAction, 'purchase') THEN h.transaction.localtransactiontax
          ELSE
          NULL
        END
          ) AS localTransactionTax
      FROM
        `airnz-ga-bigquery.125557395.ga_sessions_*`,
        UNNEST(hits) AS h
      WHERE
        _TABLE_SUFFIX BETWEEN '20190804'
        AND '20190804'
      GROUP BY
        date,
        fullVisitorId,
        visitId,
        h.hitnumber,
        source,
        visitStartTime,
        session_id,
        hostname,
        transactions,
        device.deviceCategory,
        channelGrouping,
        hostname,
        totals.visits,
        totals.timeonsite,
        totals.bounces,
        totals.newVisits )
    GROUP BY
      date,
      hostname,
      channelGrouping,
      source,
      deviceCategory,
      hostname,
      storefront )
  GROUP BY
    date,
    financialYear,
    financialQuarter,
    financialMonthNumber,
    financialWeekNumber,
    channelGrouping2,
    deviceCategory,
    storefront)
SELECT
  * EXCEPT (date_hitscope,
    financialYear_hitscope,
    financialQuarter_hitscope,
    financialMonthNumber_hitscope,
    financialWeekNumber_hitscope,
    PurchasePeriod_hitscope,
    channelGrouping2_hitscope,
    deviceCategory_hitscope,
    storefront_hitscope)
FROM (
  SELECT
    * EXCEPT (financialMonthNumber,
      financialWeekNumber),
    CASE
      WHEN financialMonthNumber IN (0, 1, 2, 3, 4, 5, 6, 7, 8, 9) THEN CONCAT('0', CAST(financialMonthNumber AS string))
    ELSE
    CAST(financialMonthNumber AS string)
  END
    AS financialMonthNumber,
    CASE
      WHEN CAST(financialWeekNumber AS int64) IN (0, 1, 2, 3, 4, 5, 6, 7, 8, 9) THEN CONCAT('0', CAST(financialWeekNumber AS string))
    ELSE
    CAST(financialWeekNumber AS string)
  END
    AS financialWeekNumber
  FROM (
    SELECT
      *
    FROM
      base_table
    WHERE
      Storefront != 'OTHER_R' ) AS base
  LEFT JOIN (
    SELECT
      *
    FROM
      hitscope_table) hitscope_table
  ON
    base.date = hitscope_table.date_hitscope
    AND base.financialYear = hitscope_table.financialYear_hitscope
    AND base.financialQuarter = hitscope_table.financialQuarter_hitscope
    AND base.financialMonthNumber = hitscope_table.financialMonthNumber_hitscope
    AND base.financialWeekNumber = hitscope_table.financialWeekNumber_hitscope
    AND base.channelGrouping2 = hitscope_table.channelGrouping2_hitscope
    AND base.deviceCategory = hitscope_table.deviceCategory_hitscope
    AND base.storefront = hitscope_table.storefront_hitscope
    AND base.PurchasePeriod = hitscope_table.PurchasePeriod_hitscope )

## Monitor BigQuery costs via Stackdriver log exports

In [ ]:
#standardSQL
WITH
  jobCompletedResponse AS (
  SELECT
    protopayload_auditlog.authenticationInfo.principalEmail AS email,
    timestamp,
    protopayload_auditlog.servicedata_v1_bigquery.jobQueryRequest.query as query,
    # Concatenate all of the project+dataset+table IDs into one long list.
    # The "\n" delimiter formats one project-dataset-table ID per line.
    # !! IMPORTANT !!
    # In BigQuery's web interface, the newline is shown as a space instead,
    # but when the results are downloaded as CSV, the newlines are exported correctly.
    ARRAY_TO_STRING(ARRAY(
      SELECT
        CONCAT(r.projectId, ".", r.datasetId, ".", r.tableId)
      FROM
        UNNEST(protopayload_auditlog.servicedata_v1_bigquery.jobCompletedEvent.job.jobStatistics.referencedTables) AS r), "\n") AS projectDatasetTableIds,
    protopayload_auditlog.servicedata_v1_bigquery.jobCompletedEvent.job.jobStatistics.totalBilledBytes
  FROM
    `GCP_Logging_BigQuery_All_logs.cloudaudit_googleapis_com_data_access_*`
  WHERE
    STARTS_WITH(_table_suffix, '201909')
    AND protopayload_auditlog.authenticationInfo.principalEmail IS NOT NULL
    AND protopayload_auditlog.servicedata_v1_bigquery.jobCompletedEvent.eventName = 'query_job_completed'
    AND protopayload_auditlog.servicedata_v1_bigquery.jobCompletedEvent.job.jobStatistics.totalBilledBytes IS NOT NULL)
    
    select 
    email,
projectDatasetTableIds,
query,
    sum(estimatedUSDCost) estimatedUSDCost from (
SELECT
  email,
  EXTRACT(YEAR FROM timestamp) AS queryYear,
  EXTRACT(MONTH FROM timestamp) AS queryMonth,
  5.0 * totalBilledBytes/POWER(2, 40) AS estimatedUSDCost,
  projectDatasetTableIds, 
  query,
  timestamp
FROM
  jobCompletedResponse
  where projectDatasetTableIds = 'airnz-ga-bigquery.Master_Reporting_Data.revenue_management_dashboard_cluster_partition'
 # and email = '!DigitalAnalytics&Optimization@airnz.co.nz'
ORDER BY
  estimatedUSDCost DESC
  )
  group by  email, projectDatasetTableIds, query
  order by estimatedUSDCost desc

## Query table metadata to find tables older than certain time

In [ ]:
#standardSQL
SELECT 
project_id, dataset_id, table_id, TIMESTAMP_MILLIS(creation_time) created_date
FROM `125557395.__TABLES__` 
where date(TIMESTAMP_MILLIS(creation_time)) < date_sub(date(current_timestamp()), interval 3 year )

## Query to check whether a table exists

In [ ]:
#standardSQL
SELECT 
project_id, dataset_id, table_id
FROM `Master_Reporting_Data.__TABLES__`  
where table_id = 'revenue_management_dashboard_cluster_partition_helper'

## Query to find top users of BigQuery

In [ ]:
#standardSQL
WITH
  jobCompletedResponse AS (
  SELECT
    protopayload_auditlog.authenticationInfo.principalEmail AS email,
    timestamp,
    # Concatenate all of the project+dataset+table IDs into one long list.
    # The "\n" delimiter formats one project-dataset-table ID per line.
    # !! IMPORTANT !!
    # In BigQuery's web interface, the newline is shown as a space instead,
    # but when the results are downloaded as CSV, the newlines are exported correctly.
    ARRAY_TO_STRING(ARRAY(
      SELECT
        CONCAT(r.projectId, ".", r.datasetId, ".", r.tableId)
      FROM
        UNNEST(protopayload_auditlog.servicedata_v1_bigquery.jobCompletedEvent.job.jobStatistics.referencedTables) AS r), "\n") AS projectDatasetTableIds,
    protopayload_auditlog.servicedata_v1_bigquery.jobCompletedEvent.job.jobStatistics.totalBilledBytes
  FROM
    `GCP_Logging_BigQuery_All_logs.cloudaudit_googleapis_com_data_access_*`
  WHERE
    --STARTS_WITH(_table_suffix, '201901')
    protopayload_auditlog.authenticationInfo.principalEmail IS NOT NULL
    AND protopayload_auditlog.servicedata_v1_bigquery.jobCompletedEvent.eventName = 'query_job_completed'
    AND protopayload_auditlog.servicedata_v1_bigquery.jobCompletedEvent.job.jobStatistics.totalBilledBytes IS NOT NULL)
SELECT
  email,
  EXTRACT(YEAR
  FROM
    timestamp) AS queryYear,
  EXTRACT(MONTH
  FROM
    timestamp) AS queryMonth,
  5.0 * totalBilledBytes/POWER(2, 40) AS estimatedUSDCost,
  projectDatasetTableIds
FROM
  jobCompletedResponse
ORDER BY
  estimatedUSDCost DESC

## Setting expiration date for a table

In [ ]:
 - refer to appscript folder, revenue management dashboard partition

## Clustering and partitioning for a table

In [ ]:
 - refer to appscript folder, revenue management dashboard partition

## Sequential Segments

### Sequential Segment Query - Followed By, Session Based

In [ ]:
#standardsql
with
touch1 as 
(
SELECT  
fullVisitorId,
CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
device.deviceCategory as deviceCat,
hits.page.pagePath as page,
 hits.type as type, 
 hits.hitnumber as hitnumber,
hits.isInteraction as interaction, 
visitStartTime+hits.time as EventTime,
hits.eventInfo.eventCategory as EventCategory,
    hits.eventInfo.eventAction as eventAction,
    hits.eventInfo.eventLabel as eventLabel
FROM `airnz-ga-bigquery.125557395.ga_sessions_*` as t, 
UNNEST(hits) as hits
WHERE 
      _TABLE_SUFFIX BETWEEN '20181001'
  AND '20181031'
  and hits.page.pagepath = '/vmanage/actions/disrupt'
),
touch2 as 
(
SELECT  
fullVisitorId,
CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
hits.page.pagePath as page,
 hits.type as type, 
 hits.hitnumber as hitnumber,
hits.isInteraction as interaction, 
visitStartTime+hits.time as EventTime,
hits.eventInfo.eventCategory as EventCategory,
    hits.eventInfo.eventAction as eventAction,
    hits.eventInfo.eventLabel as eventLabel
FROM `airnz-ga-bigquery.125557395.ga_sessions_*` as t, 
UNNEST(hits) as hits
WHERE 
      _TABLE_SUFFIX BETWEEN '20181001'
  AND '20181031'
  and hits.page.pagepath = '/vmanage/actions/disrupt/keep-flight-confirmation'

), touch3 as 
(
SELECT  
fullVisitorId,
CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
device.deviceCategory as deviceCat,
hits.page.pagePath as page,
 hits.type as type, 
 hits.hitnumber as hitnumber,
hits.isInteraction as interaction, 
visitStartTime+hits.time as EventTime,
hits.eventInfo.eventCategory as EventCategory,
    hits.eventInfo.eventAction as eventAction,
    hits.eventInfo.eventLabel as eventLabel
FROM `airnz-ga-bigquery.125557395.ga_sessions_*` as t, 
UNNEST(hits) as hits
WHERE 
      _TABLE_SUFFIX BETWEEN '20181001'
  AND '20181031'
  and hits.page.pagepath = '/vmanage/actions/disrupt'
)


SELECT
a.deviceCat,
count(distinct a.visitid) as sessions,
count(distinct a.fullvisitorid) as users
FROM
  (touch1 a
JOIN touch2 b ON a.fullVisitorId=b.fullVisitorId AND a.visitId = b.visitId AND a.eventtime < b.eventtime) 
JOIN touch3 c ON b.fullVisitorId=c.fullVisitorId AND b.visitId = c.visitId AND b.eventtime < c.eventtime
group by 1

### Sequential Segment Query - Followed By, User Based

In [ ]:
WITH
  touch1 AS (
  SELECT
    fullVisitorId,
    CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
    device.deviceCategory AS deviceCat,
    hits.page.pagePath AS page,
    hits.type AS type,
    hits.isInteraction AS interaction,
    visitStartTime+hits.time AS EventTime,
    hits.eventInfo.eventCategory AS EventCategory,
    hits.eventInfo.eventAction AS eventAction,
    hits.eventInfo.eventLabel AS eventLabel
  FROM
    `airnz-ga-bigquery.125557395.ga_sessions_*` AS t,
    UNNEST(hits) AS hits
  WHERE
    _TABLE_SUFFIX BETWEEN '20181101'
    AND '20181110'
    AND hits.page.pagepath = '/vmanage/actions/disrupt' ),
  touch2 AS (
  SELECT
    fullVisitorId,
    CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
    device.deviceCategory AS deviceCat,
    hits.page.pagePath AS page,
    hits.type AS type,
    hits.isInteraction AS interaction,
    visitStartTime+hits.time AS EventTime,
    hits.eventInfo.eventCategory AS EventCategory,
    hits.eventInfo.eventAction AS eventAction,
    hits.eventInfo.eventLabel AS eventLabel
  FROM
    `airnz-ga-bigquery.125557395.ga_sessions_*` AS t,
    UNNEST(hits) AS hits
  WHERE
    _TABLE_SUFFIX BETWEEN '20181101'
    AND '20181110'
    AND hits.page.pagepath = '/vmanage/actions/disrupt/change-flight-confirmation' )
SELECT
  device.deviceCategory,
  COUNT(DISTINCT fullVisitorId) AS users,
  COUNT(DISTINCT CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string))) AS sessions
FROM
  `airnz-ga-bigquery.125557395.ga_sessions_*` AS t,
  UNNEST(hits) AS hits
WHERE
  _TABLE_SUFFIX BETWEEN '20181101'
  AND '20181110'
  AND fullvisitorid IN(
  SELECT
    a.fullvisitorid
  FROM
    touch1 a
  JOIN
    touch2 b
  ON
    a.fullVisitorId=b.fullVisitorId AND
    #a.visitId = b.visitId and
    a.eventtime < b.eventtime)
GROUP BY
  device.deviceCategory

### Sequential Segment Query - Immediately Follows, Session Based

In [ ]:
#standardsql
with
touch1 as 
(
SELECT  
fullVisitorId,
CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
device.deviceCategory as deviceCat,
hits.page.pagePath as page,
 hits.type as type, 
 hits.hitnumber as hitnumber,
hits.isInteraction as interaction, 
visitStartTime+hits.time as EventTime,
hits.eventInfo.eventCategory as EventCategory,
    hits.eventInfo.eventAction as eventAction,
    hits.eventInfo.eventLabel as eventLabel
FROM `airnz-ga-bigquery.125557395.ga_sessions_*` as t, 
UNNEST(hits) as hits
WHERE 
      _TABLE_SUFFIX BETWEEN '20181101'
  AND '20181105'
  and hits.page.pagepath = '/vmanage/actions/disrupt'
),
touch2 as 
(
SELECT  
fullVisitorId,
CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
hits.page.pagePath as page,
 hits.type as type, 
 hits.hitnumber as hitnumber,
hits.isInteraction as interaction, 
visitStartTime+hits.time as EventTime,
hits.eventInfo.eventCategory as EventCategory,
    hits.eventInfo.eventAction as eventAction,
    hits.eventInfo.eventLabel as eventLabel
FROM `airnz-ga-bigquery.125557395.ga_sessions_*` as t, 
UNNEST(hits) as hits
WHERE 
      _TABLE_SUFFIX BETWEEN '20181101'
  AND '20181105'
  and hits.page.pagepath = '/vmanage/actions/disrupt/keep-flight-confirmation'

)


select 
a.deviceCat,
count(distinct a.visitId) sessions,
count(distinct a.fullvisitorid) users
from touch1 a
join touch2 b on
a.fullVisitorId=b.fullVisitorId and 
a.visitId = b.visitId 
where (b.hitnumber - a.hitnumber) = 1
group by 1

### Sequential Segment Query - Immediately Follows, User Based

In [ ]:
#standardsql
with
touch1 as 
(
SELECT  
fullVisitorId,
CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
device.deviceCategory as deviceCat,
hits.page.pagePath as page,
 hits.type as type, 
 hits.hitnumber as hitnumber,
hits.isInteraction as interaction, 
visitStartTime+hits.time as EventTime,
hits.eventInfo.eventCategory as EventCategory,
    hits.eventInfo.eventAction as eventAction,
    hits.eventInfo.eventLabel as eventLabel
FROM `airnz-ga-bigquery.125557395.ga_sessions_*` as t, 
UNNEST(hits) as hits
WHERE 
      _TABLE_SUFFIX BETWEEN '20181101'
  AND '20181130'
  and hits.page.pagepath = '/vmanage/actions/disrupt'
),
touch2 as 
(
SELECT  
fullVisitorId,
CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string)) AS visitId,
hits.page.pagePath as page,
 hits.type as type, 
 hits.hitnumber as hitnumber,
hits.isInteraction as interaction, 
visitStartTime+hits.time as EventTime,
hits.eventInfo.eventCategory as EventCategory,
    hits.eventInfo.eventAction as eventAction,
    hits.eventInfo.eventLabel as eventLabel
FROM `airnz-ga-bigquery.125557395.ga_sessions_*` as t, 
UNNEST(hits) as hits
WHERE 
      _TABLE_SUFFIX BETWEEN '20181101'
  AND '20181130'
  and hits.page.pagepath = '/vmanage/actions/disrupt/keep-flight-confirmation'

)

select 
device.deviceCategory as device, 
count(distinct fullvisitorid) as users,
count(distinct CONCAT(fullVisitorId, '-', CAST(visitStartTime AS string))) as sessions
FROM `airnz-ga-bigquery.125557395.ga_sessions_*` as t, 
UNNEST(hits) as hits
WHERE 
      _TABLE_SUFFIX BETWEEN '20181101'
  AND '20181130' and fullvisitorid in(
select 
a.fullvisitorid
from touch1 a
join touch2 b on
a.fullVisitorId=b.fullVisitorId and 
a.visitId = b.visitId 
where (b.hitnumber - a.hitnumber) = 1
)
group by 1

## Analysis for Optimizely A/B testing impact

In [ ]:
WITH
  minvisits_table AS (

  SELECT
    fullvisitorid,
    dimension74 AS dim74,
    MIN(visitnumber) AS initialvisit
  FROM (
    SELECT
      fullvisitorid,
      visitid,
      date,
      visitnumber,
      dimension74,
      SUM(flightbooking) AS fb,
      SUM(flightsearch) AS fs
    FROM (
      SELECT
        fullvisitorid,
        visitid,
        visitnumber,
        date,
        hits.hitNumber,
        hits.page.pagePath,
        (
        SELECT
          value
        FROM
          t.customDimensions
        WHERE
          index=74) AS dimension74,
        CASE
          WHEN REGEXP_CONTAINS(hits.page.pagePath, r'^/vbook/actions/(mobi/|)bookingconfirmation') AND hits.type = 'PAGE' THEN 1
          ELSE 0
        END AS flightBooking,
        CASE
          WHEN REGEXP_CONTAINS(hits.page.pagePath,r'^/vbook/actions/(selectflights|selectitinerary|(mobi/|)createitinerary)') AND hits.type = 'PAGE' THEN 1
          ELSE 0
        END AS flightSearch
      FROM
        `airnz-ga-bigquery.125557395.ga_sessions_*`t,
        UNNEST(hits) AS hits
      WHERE
        _TABLE_SUFFIX BETWEEN '20180611'
        AND '20180728')
    WHERE
      dimension74 IS NOT NULL
      AND REGEXP_CONTAINS(dimension74,"10802251481|10800260542|10800697990" )
    GROUP BY
      1,
      2,
      3,
      4,
      5)
  GROUP BY
    1,2)
    
select 
#fullvisitorid_base, visitid, 
dim74, sum(sessions) as sessions, sum(fs) as fs, sum(fb) as fb from( 
SELECT
  *
FROM (
  SELECT
    *
  FROM (
    SELECT
      fullvisitorid AS fullvisitorid_base,
      visitnumber,
      dimension74,
      count(distinct visitid) as sessions,
      SUM(flightbooking) AS fb,
      SUM(flightsearch) AS fs
    FROM (
      SELECT
        fullvisitorid,
        visitid,
        visitnumber,
        date,
        (SELECT value FROM t.customDimensions WHERE index=74) AS dimension74,
        CASE WHEN REGEXP_CONTAINS(hits.page.pagePath, r'^/vbook/actions/(mobi/|)bookingconfirmation') AND hits.type = 'PAGE' THEN 1
          ELSE 0 END AS flightBooking,
        CASE WHEN REGEXP_CONTAINS(hits.page.pagePath,r'^/vbook/actions/(selectflights|selectitinerary|(mobi/|)createitinerary)') AND hits.type = 'PAGE' THEN 1
          ELSE 0 END AS flightSearch
      FROM
        `airnz-ga-bigquery.125557395.ga_sessions_*`t,
        UNNEST(hits) AS hits
      WHERE
        _TABLE_SUFFIX BETWEEN '20180611'
        AND '20180728' 
        )
    GROUP BY
      1,
      2,
      3) AS base
  LEFT JOIN
    minvisits_table
  ON
    base.fullvisitorid_base = minvisits_table.fullvisitorid
    AND base.visitnumber >= minvisits_table.initialvisit)
WHERE
  fullvisitorid IS NOT NULL)
group by 1

In [ ]:
## Backfilling using First Value

In [ ]:
#standardSQL
CREATE TEMP FUNCTION
  customDimensionByIndex(indx INT64,
    arr ARRAY<STRUCT<index INT64,
    value STRING>>) AS ( (
    SELECT
      x.value
    FROM
      UNNEST(arr) x
    WHERE
      indx=x.index) );

  SELECT
    ProductName,
    destination_city_backfilled,
    destination_country_backfilled,
    AncillaryCat,
    SUM(CASE
        WHEN REGEXP_CONTAINS(booking_lead_days,r'undefined') THEN CAST(0 AS INT64)
        WHEN booking_lead_days IS NULL THEN CAST(0 AS INT64)
        WHEN CAST(booking_lead_days AS INT64) < 0 THEN CAST(0 AS INT64)
        WHEN CAST(booking_lead_days AS INT64) > 364 THEN CAST(0 AS INT64)
        ELSE CAST(booking_lead_days AS INT64) END) AS bldsum,
    COUNT (visitId) AS totalcount
  FROM (
    SELECT
      date,
      fullvisitorId,
      visitId,
      transactionId,
      v2ProductName AS ProductName,
      destination_city,
      destination_country,
      booking_lead_days,
      IFNULL(FIRST_VALUE(destination_city IGNORE NULLS) OVER (PARTITION BY CAST(fullvisitorId AS STRING)
        ORDER BY
          CAST(visitId AS STRING) ASC,
          time ASC ROWS BETWEEN UNBOUNDED PRECEDING
          AND CURRENT ROW ),
        FIRST_VALUE(destination_city IGNORE NULLS) OVER (PARTITION BY CAST(fullvisitorId AS STRING)
        ORDER BY
          CAST(visitId AS STRING) DESC,
          time DESC ROWS BETWEEN CURRENT ROW
          AND UNBOUNDED FOLLOWING)) AS destination_city_backfilled,
      IFNULL(FIRST_VALUE(destination_country IGNORE NULLS) OVER (PARTITION BY CAST(fullvisitorId AS STRING)
        ORDER BY
          CAST(visitId AS STRING) ASC,
          time ASC ROWS BETWEEN UNBOUNDED PRECEDING
          AND CURRENT ROW),
        FIRST_VALUE(destination_country IGNORE NULLS) OVER (PARTITION BY CAST(fullvisitorId AS STRING)
        ORDER BY
          CAST(visitId AS STRING) DESC,
          time DESC ROWS BETWEEN CURRENT ROW
          AND UNBOUNDED FOLLOWING)) AS destination_country_backfilled
    FROM (
      SELECT
        --- Session Information ---
        date,
        h.time,
        fullvisitorId,
        CONCAT(fullVisitorId,"-",CAST(visitID AS STRING)) AS visitId,
        h.transaction.transactionId,
        p.v2ProductName,
        --- Custom Dimension ---
        customDimensionByIndex(30,
          p.customDimensions) AS destination_city,
        customDimensionByIndex(31,
          p.customDimensions) AS destination_country,
        customDimensionByIndex(37,
          p.customDimensions) AS booking_lead_days
      FROM
        `airnz-ga-bigquery.125557395.ga_sessions_*`t,
        UNNEST (hits) AS h,
        UNNEST(h.product) AS p
      WHERE
        _TABLE_SUFFIX BETWEEN '20180101'
        AND '20181031'
        AND h.eCommerceAction.action_type = "6"
        AND NOT REGEXP_CONTAINS(v2ProductName,r'credit card fee|product upgrade|seat select|pre booking|multi currency fee|delivery fee|flexible upgrade|extra bags|fare hold|flexipay fee|held flight fare|additional driver|ldw|unaccompanied')
        AND fullvisitorId IS NOT NULL
      GROUP BY
        date,
        time,
        visitid,
        fullvisitorID,
        transactionId,
        v2ProductName,
        destination_city,
        destination_country,
        booking_lead_days
        -- HAVING
        --  NOT REGEXP_CONTAINS(v2ProductName,r'infant|adult|child')
        )
    GROUP BY
      date,
      time,
      fullvisitorId,
      visitId,
      transactionId,
      ProductName,
      destination_city,
      destination_country,
      booking_lead_days
      --    HAVING
      --      REGEXP_CONTAINS(fullVisitorId,r'955079343795455209')
      )
  LEFT JOIN
    `guid_rollup.ancillary_mapping2`
  USING
    (ProductName)
  GROUP BY
    1,
    2,
    3,
    4
  HAVING
    NOT REGEXP_CONTAINS(ProductName,r'infant|adult|child')
    AND destination_city_backfilled IS NOT NULL
    
 --   AND REGEXP_CONTAINS(destination_city_backfilled,r'per')
 
  --ORDER BY
  --  ProductName DESC
  --HAVING AncillaryCat IS NULL